In [2]:
#Import FASTA file (Get hg38 fasta file)
import os, subprocess

if not os.path.isfile('data/hg19.ml.fa'):
    subprocess.call('curl -o data/hg19.ml.fa https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa', shell=True)
    subprocess.call('curl -o data/hg19.ml.fa.fai https://storage.googleapis.com/basenji_tutorial_data/hg19.ml.fa.fai', shell=True) 

In [3]:
#Import pre-processed data (if it is not processed, you might want to run bam_cov.py on bam file to distribute multi-mapping reads and estimate genomic coverage)

if not os.path.isfile('data/CNhs11760.bw'):
    subprocess.call('curl -o data/CNhs11760.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs11760.bw', shell=True)
    subprocess.call('curl -o data/CNhs12843.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs12843.bw', shell=True)
    subprocess.call('curl -o data/CNhs12856.bw https://storage.googleapis.com/basenji_tutorial_data/CNhs12856.bw', shell=True)


In [9]:
 import basenji

In [3]:
#Import bigwigfile 
#import pysam
#bamfile = pysam.AlignmentFile("/users/ysuita/data/ysuita/basenji/data/GSC1-stem.bam", "rb")
#print(bamfile)
Bigwig=""

#Import hg38 fasta file
from pysam import FastaFile
fasta="/gpfs/data/shared/databases/refchef_refs/grch38_release98/primary/Homo_sapiens.GRCh38.dna.primary_assembly.fa"
 # read FASTA file
#sequences_object = FastaFile(fasta)
print(fasta)

/gpfs/data/shared/databases/refchef_refs/grch38_release98/primary/Homo_sapiens.GRCh38.dna.primary_assembly.fa


In [2]:
#Write out the above bigwig files and lables to a sample table

lines = [['index','identifier','file','clip','sum_stat','description']]
lines.append(['0', 'CNhs11760', 'data/CNhs11760.bw', '384', 'sum', 'aorta'])
lines.append(['1', 'CNhs12843', 'data/CNhs12843.bw', '384', 'sum', 'artery'])
lines.append(['2', 'CNhs12856', 'data/CNhs12856.bw', '384', 'sum', 'pulmonic_valve'])

samples_out = open('data/heart_wigs.txt', 'w')
for line in lines:
    print('\t'.join(line), file=samples_out)
samples_out.close()


In [20]:
#Run model - choose genomic sequences to form batchs for stochastic gradient descent, devide them into training (80%)/valudation(10%)/test sets(10%), and construct TFRecords to provide to downstream programs
#Try 70% / 15% / 15% 
import basenji
! bin/basenji_data.py -d .1 -g /users/ysuita/data/ysuita/basenji/tutorials/data/unmap_macro.bed -l 131072 --local -o /users/ysuita/data/ysuita/basenji/data/heart_l131k -p 8 -t .1 -v .1 -w 128 /users/ysuita/data/ysuita/basenji/data/hg19.ml.fa /users/ysuita/data/ysuita/basenji/data/heart_wigs.txt

stride_train 1 converted to 131072.000000
stride_test 1 converted to 131072.000000
Contigs divided into
 Train:  4701 contigs, 2169074921 nt (0.8005)
 Valid:   572 contigs,  270358978 nt (0.0998)
 Test:    584 contigs,  270330829 nt (0.0998)
/users/ysuita/data/ysuita/basenji/bin/basenji_data_read.py -w 128 -u sum -c 384.000000 -s 1.000000 data/CNhs11760.bw /users/ysuita/data/ysuita/basenji/data/heart_l131k/sequences.bed /users/ysuita/data/ysuita/basenji/data/heart_l131k/seqs_cov/0.h5
/users/ysuita/data/ysuita/basenji/bin/basenji_data_read.py -w 128 -u sum -c 384.000000 -s 1.000000 data/CNhs12843.bw /users/ysuita/data/ysuita/basenji/data/heart_l131k/sequences.bed /users/ysuita/data/ysuita/basenji/data/heart_l131k/seqs_cov/1.h5
/users/ysuita/data/ysuita/basenji/bin/basenji_data_read.py -w 128 -u sum -c 384.000000 -s 1.000000 data/CNhs12856.bw /users/ysuita/data/ysuita/basenji/data/heart_l131k/sequences.bed /users/ysuita/data/ysuita/basenji/data/heart_l131k/seqs_cov/2.h5
/users/ysuita/dat

In [23]:
#To confirm data/heart_l131k contains relevant data for training
! cut -f4 data/heart_l131k/sequences.bed | sort | uniq -c

    179 test
   1499 train
    180 valid


In [24]:
#To confirm sequences.bed contains the train/valid/test sequences.
! head -n3 data/heart_l131k/sequences.bed
! grep valid data/heart_l131k/sequences.bed | head -n3
! grep test data/heart_l131k/sequences.bed | head -n3
! ls -l data/heart_l131k/tfrecords/*.tfr

chr2	140425791	140556863	train
chr16	27143973	27275045	train
chr14	72972403	73103475	train
chr11	116271608	116402680	valid
chr2	26239628	26370700	valid
chr6	102241368	102372440	valid
chr14	76007464	76138536	test
chr13	99391330	99522402	test
chr15	52280212	52411284	test
-rw-r--r-- 1 ysuita ntapinos 10609221 Jul 26 15:19 data/heart_l131k/tfrecords/test-0.tfr
-rw-r--r-- 1 ysuita ntapinos 15168689 Jul 26 15:20 data/heart_l131k/tfrecords/train-0.tfr
-rw-r--r-- 1 ysuita ntapinos 15153640 Jul 26 15:20 data/heart_l131k/tfrecords/train-1.tfr
-rw-r--r-- 1 ysuita ntapinos 15175334 Jul 26 15:20 data/heart_l131k/tfrecords/train-2.tfr
-rw-r--r-- 1 ysuita ntapinos 15181231 Jul 26 15:20 data/heart_l131k/tfrecords/train-3.tfr
-rw-r--r-- 1 ysuita ntapinos 15151442 Jul 26 15:20 data/heart_l131k/tfrecords/train-4.tfr
-rw-r--r-- 1 ysuita ntapinos 12976797 Jul 26 15:19 data/heart_l131k/tfrecords/train-5.tfr
-rw-r--r-- 1 ysuita ntapinos 10661284 Jul 26 15:19 data/heart_l131k/tfrecords/valid-0.tfr


In [3]:
#For training, grab HDF5 file which contains 10% of training sequences

import glob, os, subprocess
from IPython.display import IFrame

if len(glob.glob('data/heart_l131k/tfrecords/*.tfr')) == 0:
    subprocess.call('curl -o data/heart_l131k.tgz https://storage.googleapis.com/basenji_tutorial_data/heart_l131k.tgz', shell=True)
    subprocess.call('tar -xzvf data/heart_l131k.tgz', shell=True)

In [9]:
#For Training
output="/users/ysuita/data/ysuita/basenji/tutorials/models/heart"
json="/users/ysuita/data/ysuita/basenji/tutorials/models/params_small.json"
data_dir="/users/ysuita/data/ysuita/basenji/data/heart_l131k"
! /users/ysuita/data/ysuita/basenji/bin/basenji_train.py -o  output json data_dir


2022-07-26 16:05:06.065344: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/runtime/opt/gcc/8.3/lib64:/gpfs/runtime/opt/gsl/2.5/lib:/gpfs/runtime/opt/boost/1.69/lib:/gpfs/runtime/opt/R/4.0.0/lib64/R/lib:/gpfs/runtime/opt/pcre2/10.35/lib:/gpfs/runtime/opt/python/3.8.12_gcc8.3/lib:/gpfs/runtime/opt/intel/2017.0/lib/intel64:/gpfs/runtime/opt/intel/2017.0/mkl/lib/intel64:/gpfs/runtime/opt/java/8u111/jre/lib/amd64
2022-07-26 16:05:06.065391: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-07-26 16:05:15.528435: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /gpfs/runtime

In [4]:
#For testing gene expression precitions based on the trained model

import json
import h5py


test_parameters="/users/ysuita/data/ysuita/basenji/tutorials/models/params_small.json"
#Read model parameters 
with open(test_parameters) as params_open:
 test_params=json.load(params_open)
print(test_params)

trained_model="/users/ysuita/data/ysuita/basenji/tutorials/models/heart/model_best.h5"
#Read trained model
with open(trained_model) as train: 
 tra_model=h5py.File(trained_model,'r')
print(tra_model)

test_output="/users/ysuita/data/ysuita/basenji/tutorials/models/heart_test"
test_inputdata="/users/ysuita/data/ysuita/basenji/data/heart_l131k"

#! echo testing
! nvcc --version
#! /users/ysuita/data/ysuita/basenji/bin/basenji_test.py --ai 0,1,2 -o test_output --rc --shifts "1,0,-1" test_params train.tra_model test_inputdata
! /users/ysuita/data/ysuita/basenji/bin/basenji_test.py --ai 0,1,2 -o /users/ysuita/data/ysuita/basenji/tutorials/models/heart_test --rc --shifts "1,0,-1" test_params train.tra_model /users/ysuita/data/ysuita/basenji/data/heart_l131k


#print(test_targets_ti_log)
#print(test_preds_ti_log)

{'train': {'batch_size': 4, 'loss': 'poisson', 'optimizer': 'sgd', 'learning_rate': 0.1, 'momentum': 0.99, 'train_epochs': 20, 'patience': 8, 'clip_norm': 2}, 'model': {'seq_length': 131072, 'augment_rc': True, 'augment_shift': 3, 'activation': 'gelu', 'norm_type': 'batch', 'bn_momentum': 0.9, 'trunk': [{'name': 'conv_block', 'filters': 64, 'kernel_size': 15, 'pool_size': 8}, {'name': 'conv_tower', 'filters_init': 64, 'filters_mult': 1.125, 'kernel_size': 5, 'pool_size': 4, 'repeat': 2}, {'name': 'dilated_residual', 'filters': 32, 'rate_mult': 2, 'repeat': 6, 'dropout': 0.25}, {'name': 'conv_block', 'filters': 64, 'dropout': 0.05}], 'head': {'name': 'final', 'units': 3, 'activation': 'softplus'}}}
<HDF5 file "model_best.h5" (mode r)>
/bin/bash: nvcc: command not found
2022-07-28 16:54:08.861605: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or d

In [16]:
import pandas as pd
from basenji import plots
z=pd.read_pickle("/users/ysuita/data/ysuita/basenji/data/z.pkl")
print(z)
plots.regplot(
          x='x',
          y='y',
          data=z,
          poly_order=1,
          alpha=0.3,
          sample=500)

          #figsize=(6, 6),
          #x_label='log2 Experiment',
          #y_label='log2 Prediction',
          #table=True)

              x         y
0      0.245256  0.223097
1      0.000000  0.049721
2      0.449407  0.077233
3      0.000000  0.131214
4      0.000000  0.165300
...         ...       ...
22907  0.000000  0.451469
22908  0.000000  0.664447
22909  0.000000  1.041659
22910  1.128639  1.758223
22911  0.000000  0.318966

[22912 rows x 2 columns]


TypeError: regplot() got an unexpected keyword argument 'x'